# Recursive Inverted Index (5 Points)

Copyright [Big Data Analytics Group](https://bigdata.uni-saarland.de/), [CC-BY-SA](https://creativecommons.org/licenses/by-sa/4.0/legalcode)

In this exercise, you will implement an [inverted index](https://en.wikipedia.org/wiki/Inverted_index). An inverted index is a mapping from the content (e.g. words) of
a document to the document itself. In addition, in this exercise we radix-partition the words, i.e. we _recursively_ partition the words characterwise similar to the notebook [Indexing by Recursive External Partitioning.ipynb](https://github.com/BigDataAnalyticsGroup/bigdataengineering/blob/master/Indexing%20by%20Recursive%20External%20Partitioning.ipynb). This structure allows us to easily search for documents that contain a word that exactly matches a given search word, but also words that start with the given search word (prefix search). Implement the following functions of the class `RecursiveInvertedIndex` in the attached notebook:
* `__init__`: This is the constructor of the class, which takes a list of documents names and creates the
    recursive, inverted index based on these documents. Your words must be radix-partitioned. You can use the helper
        function`get_words_from_document(document_name)`, which takes a document name as input and then
        returns a set of words contained in the document. Note, that all words are converted to lowercase.
* `search(word)`: This function uses the inverted index to return a list of all matching documents given the key `word`. Matching here means that the search word either exactly matches one of the words in a document, or at least one of the words in a document starts with the search word. If no documents exist for a search word, an empty set is returned. Otherwise, the set of all matching documents is returned. Your implementation should be case-insensitive.

Your implementation must pass all provided unit tests without hardcoding!

In [19]:
def get_words_from_document(document_name):
    with open(document_name) as f:
        text = f.read().replace('.', '')
        word_list = text.split(" ")
        words = set([word.lower() for word in word_list])
        return words

In [84]:
# Upload the contents of this cell to our CMS as a text file

class Structure:
    def __init__(self):
        self.indices = set()
        self.children = {}

    def update(self, string, index):
        # print('Adding', string)
        self.indices.add(index)
        if len(string) == 0:
            return len(self.indices)

        # expand children
        if self.children.get('a') is None:
            for i in range(26):
                self.children[chr(97+i)] = Structure()


        return self.children[string[0]].update(string[1:], index)

    def start_search(self, string):
        if string == '':
            return set()
        return self.search(string)

    def search(self, string):
        if len(string) == 0:
            return self.indices

        if len(self.children) == 0:
            return set()

        if self.children.get(string[0]) is None:
            return set()

        return self.children[string[0]].search(string[1:])

    def __str__(self):
        return f"{{indices: {self.indices}, {self.children}}}"



# A class implementing a recursive, inverted index
class RecursiveInvertedIndex:
    # document_names: Names of the documents to be put in the index
    def __init__(self, document_names):
        # Add your code here!
        # ...
        # You should initialise your recursive, inverted index here. Note that your words must be radix-partitioned.

        self.document_names = document_names
        self.data = Structure()
        self.init_structure()

    def init_structure(self):
        # create data structure
        for i, name in enumerate(self.document_names):
            for word in get_words_from_document(name):
                self.data.update(word, name)

    def search(self, word): 
        # Add your code here!
        # ...
        # This function should return the set of all matching documents. A matching document either contains exactly the search word or
        # a word that starts with the given search word (or both).
        return self.data.start_search(word.lower())

In [85]:
documents = ["data/indexes/test_file_1.txt", "data/indexes/test_file_2.txt", "data/indexes/test_file_3.txt"]
rec_inverted_index = RecursiveInvertedIndex(documents)

In [86]:
search_result = rec_inverted_index.search("I")
# ("data/indexes/test_file_1.txt" in search_result and "data/indexes/test_file_3.txt" in search_result)
search_result


{'data/indexes/test_file_1.txt',
 'data/indexes/test_file_2.txt',
 'data/indexes/test_file_3.txt'}

### Unit tests

Note that test cases are by no means exhaustive!

In [87]:
import unittest

class RecursiveInvertedIndexTest(unittest.TestCase):
    
    def setUp(self):
        self.documents = ["data/indexes/test_file_1.txt", "data/indexes/test_file_2.txt", "data/indexes/test_file_3.txt"]
        self.rec_inverted_index = RecursiveInvertedIndex(self.documents)

    def test_no_result(self):
        search_result = self.rec_inverted_index.search("Im")
        self.assertEqual(len(search_result), 0)
        search_result = self.rec_inverted_index.search("BDER")
        self.assertEqual(len(search_result), 0)
        search_result = self.rec_inverted_index.search("lotin")
        self.assertEqual(len(search_result), 0)
        search_result = self.rec_inverted_index.search("")
        self.assertEqual(len(search_result), 0)
        search_result = self.rec_inverted_index.search(" ")
        self.assertEqual(len(search_result), 0)
        search_result = self.rec_inverted_index.search("ar")
        self.assertEqual(len(search_result), 0)
    
    def test_with_result(self):
        search_result = self.rec_inverted_index.search("bde")
        self.assertEqual(len(search_result), 2)
        self.assertTrue("data/indexes/test_file_1.txt" in search_result and "data/indexes/test_file_3.txt" in search_result)
        search_result = self.rec_inverted_index.search("DBS")
        self.assertEqual(len(search_result), 2)
        self.assertTrue("data/indexes/test_file_2.txt" in search_result and "data/indexes/test_file_3.txt" in search_result)
        search_result = self.rec_inverted_index.search("I")
        self.assertEqual(len(search_result), 3)
        for document in self.documents:
            self.assertTrue(document in search_result)
        search_result = self.rec_inverted_index.search("course")
        self.assertEqual(len(search_result), 2)
        self.assertTrue("data/indexes/test_file_1.txt" in search_result and "data/indexes/test_file_2.txt" in search_result)
        search_result = self.rec_inverted_index.search("o")
        self.assertEqual(len(search_result), 1)
        self.assertTrue("data/indexes/test_file_3.txt" in search_result)

In [88]:
# Run the unit test without shutting down the jupyter kernel
unittest.main(argv=['ignored', '-v'], verbosity=2, exit=False)

test_no_result (__main__.RecursiveInvertedIndexTest) ... ok
test_with_result (__main__.RecursiveInvertedIndexTest) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.007s

OK
